In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

2024-02-12 23:13:59.589263: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_url = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"
application_df = pd.read_csv(data_url)

In [3]:
application_counts = application_df['APPLICATION_TYPE'].value_counts()
replace_applications = list(application_counts[application_counts < 500].index)
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(replace_applications, "Other")

In [4]:
application_df_encoded = pd.get_dummies(application_df, drop_first=True)

In [ ]:
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1).values
y = application_df_encoded['IS_SUCCESSFUL'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = Sequential([
    Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('optimized_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping, checkpoint])

In [ ]:
from tensorflow.keras.models import load_model
saved_model = load_model('AlphabetSoupCharity_Optimization.h5')

In [ ]:
model_loss, model_accuracy = saved_model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Optimized Model Loss: {model_loss}, Optimized Model Accuracy: {model_accuracy}")